In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy
from cfd_common import *
from math import *
import scipy.interpolate
import os

In [2]:
%matplotlib qt

In [3]:
plt.ion()

In [4]:
path_prefix = 'validation_new'
if not os.path.exists(path_prefix):
    os.mkdir(path_prefix)

In [5]:
##### Domain #####

# Domain size
Lx = pi
Ly = pi

# Physical grid size
Nx_phys = 51
Ny_phys = 51
Δx = Lx/Nx_phys
Δy = Ly/Ny_phys

# Numerical grid size, including ghost points
Nx = Nx_phys + 2
Ny = Ny_phys + 2

# Physical grid (x,y)
# Due to the "staggered-like" approach to boundary conditions with ghost points,
#  the border go through the middle of border cells
x = np.linspace(Δx/2, Lx-Δx/2, Nx_phys) 
y = np.linspace(Δy/2, Ly-Δy/2, Ny_phys)
[xx,yy] = np.meshgrid(x,y) 

In [6]:
a = 0
b = 0

def exact_solution (x, y, t):
    u =  np.sin(x) * np.cos(y) * np.cos(t)
    v = -np.sin(y) * np.cos(x) * np.cos(t)
    ρ = 1 + a * np.sin(x)**2 * np.sin(y)**2 + b * np.sin(x)**4 * np.sin(y)**4
    p = -2 * np.cos(x) * np.cos(y) * np.cos(t)
    fx = ((-np.sin(t)*np.cos(y) + np.cos(t)**2*np.cos(x))*(b*np.sin(x)**4*np.sin(y)**4 + a**np.sin(x)**2*np.sin(y)**2 + 1) + 4*np.cos(t)*np.cos(y))*np.sin(x)
    fy = (np.sin(t)*np.cos(x) + np.cos(t)**2*np.cos(y))*(b*np.sin(x)**4*np.sin(y)**4 + a*np.sin(x)**2*np.sin(y)**2 + 1)*np.sin(y)
    return ρ,u,v,p

def force (x, y, t):
    fx = ((-np.sin(t)*np.cos(y) + np.cos(t)**2*np.cos(x))*(b*np.sin(x)**4*np.sin(y)**4 + a**np.sin(x)**2*np.sin(y)**2 + 1) + 4*np.cos(t)*np.cos(y))*np.sin(x)
    fy = (np.sin(t)*np.cos(x) + np.cos(t)**2*np.cos(y))*(b*np.sin(x)**4*np.sin(y)**4 + a*np.sin(x)**2*np.sin(y)**2 + 1)*np.sin(y)
    return fx,fy

In [7]:
##### Fields #####

ex_ρ,ex_u,ex_v,ex_p = exact_solution(xx, yy, 0)

# Velocity field (u,v)[i,j], initialized to "zero"
u = np.zeros((Nx,Ny))
u[1:-1,1:-1] = np.transpose( ex_u )
v = np.zeros((Nx,Ny))
v[1:-1,1:-1] = np.transpose( ex_v )

ustar = np.zeros((Nx,Ny))
vstar = np.zeros((Nx,Ny))

# Pressure field phi[i,j] and its gradient (dx_phi,dy_phi)
phi = np.zeros((Nx,Ny))
phi[1:-1,1:-1] = np.transpose( ex_p )
dx_phi = np.zeros((Nx,Ny))
dy_phi = np.zeros((Nx,Ny))

# Density field
ρ = np.ones((Nx,Ny))
ρ[1:-1,1:-1] = np.transpose( ex_ρ )

s = np.zeros((Nx,Ny))

# Total mass
def mass_check (ρ):
    return Δx*Δy*np.sum(ρ)

In [8]:
Δt = 0.001

t = 0. # total time

µ = 1

# how much do we correct for diffusion in semi-lag advection at the price of dispersion
ρ_semilag2_coeff = 0.8

niter = 0
disp_modulo = 100

In [9]:
#data = np.load("bubble-test-200x200-t2.40237-niter240237.npz")
#u = data['u']
#v = data['v']
#ρ = data['rho']
#t = 2.40237
#niter = 240237
# np.savez("bubble-test-200x200-.npz", u=u, v=v, rho=ρ)

In [10]:
err_history_t = []
err_history_uabs = []
err_history_umean = []
err_history_vabs = []
err_history_vmean = []
err_history_ρabs = []
err_history_ρmean = []
err_history_pabs = []
err_history_pmean = []

In [11]:
fig = plt.figure(1, figsize=(14,7))

loop_continue = True
def on_close (event):
    global loop_continue
    loop_continue = False
fig.canvas.mpl_connect('close_event', on_close)

def do_plot (save_image):
    global err_history_t, err_history_uabs, err_history_umean, err_history_vabs, err_history_vmean, err_history_ρabs, err_history_ρmean, err_history_pabs, err_history_pmean
    fig.clear()
    ((ax1,ax2,ax3),(ax4,ax5,ax6)) = fig.subplots(nrows=2, ncols=3)

    def plot_vel (ax, modx=None, mody=None, vmax=3):
        if modx is None:
            modx = max(1,int(round(Nx_phys/20)))
        if mody is None:
            mody = max(1,int(round(Ny_phys/20)))
        ax.quiver(xx[::modx,::mody], yy[::modx,::mody], np.transpose(u[1:-1,1:-1])[::modx,::mody], np.transpose(v[1:-1,1:-1])[::modx,::mody], scale=vmax*5, minlength=0.001, headwidth=4, headlength=6, headaxislength=5, width=0.005)
        ax.set_aspect('equal', adjustable='box')

    fig.suptitle(r"Validation. ${}\times{}$ grid. SemiLag2({}) for $\rho$, total mass : {:.2f}, $t={:.3f}$, $\Delta t=${:.0e}.".format(Nx_phys,Ny_phys,ρ_semilag2_coeff,mass_check(ρ),t,Δt))
    
    p = np.transpose(phi[1:-1,1:-1]+µ*s[1:-1,1:-1])
    im = ax1.imshow(p-np.mean(p), origin='lower', extent=(0,Lx,0,Ly), cmap='plasma', vmin=-2.2, vmax=2.2)
    fig.colorbar(im, ax=ax1)
    ax1.axis('off')
    plot_vel(ax1)
    ax1.set_title("Pressure and velocity fields")

    ex_ρ,ex_u,ex_v,ex_p = exact_solution(xx, yy, t)
    err_u = np.transpose(u[1:-1, 1:-1]) - ex_u
    err_v = np.transpose(v[1:-1, 1:-1]) - ex_v
    err_ρ = np.transpose(ρ[1:-1, 1:-1]) - ex_ρ
    err_p = (p-np.mean(p)) - (ex_p-np.mean(ex_p))
    err_history_t += [ t ]
    err_history_uabs += [ np.max(np.abs(err_u)) ]
    err_history_umean += [ np.mean(np.abs(err_u)) ]
    err_history_vabs += [ np.max(np.abs(err_v)) ]
    err_history_vmean += [ np.mean(np.abs(err_v)) ]
    err_history_ρabs += [ np.max(np.abs(err_ρ)) ]
    err_history_ρmean += [ np.mean(np.abs(err_ρ)) ]
    err_history_pabs += [ np.max(np.abs(err_p)) ]
    err_history_pmean += [ np.mean(np.abs(err_p)) ]
    
    k = 1
    
    im = ax2.imshow(err_p, origin='lower', extent=(0,Lx,0,Ly), vmin=-k*0.2, vmax=+k*0.2, cmap='bwr')
    fig.colorbar(im, ax=ax2)
    ax2.axis('off')
    ax2.set_title("Error on pressure")
    
    im = ax3.imshow(err_ρ, origin='lower', extent=(0,Lx,0,Ly), vmin=-k*0.03, vmax=+k*0.03, cmap='bwr')
    fig.colorbar(im, ax=ax3)
    ax3.axis('off')
    ax3.set_title("Error on density")
    
    im = ax4.imshow(err_u, origin='lower', extent=(0,Lx,0,Ly), vmin=-k*0.08, vmax=+k*0.08, cmap='bwr')
    fig.colorbar(im, ax=ax4)
    ax4.axis('off')
    ax4.set_title("Error on $u$")
    
    im = ax5.imshow(err_v, origin='lower', extent=(0,Lx,0,Ly), vmin=-k*0.08, vmax=+k*0.08, cmap='bwr')
    fig.colorbar(im, ax=ax5)
    ax5.axis('off')
    ax5.set_title("Error on $v$")
    
    ax6.plot(err_history_t, err_history_uabs, label=r"$L^\infty$ err on $u$")
    ax6.plot(err_history_t, err_history_vabs, label=r"$L^\infty$ err on $v$")
    ax6.plot(err_history_t, err_history_ρabs, label=r"$L^\infty$ err on $\rho$")
    ax6.plot(err_history_t, np.array(err_history_pabs)/10, label=r"$0.1\times$ $L^\infty$ err on $p$")
#    ax6.set_ylim(0, 0.003)
    ax6.set_xlim(0, 3*pi)
    ax6.axvline(  pi/2, linestyle='--', color='gray', lw=1)
    ax6.axvline(  pi  , linestyle='-' , color='gray', lw=1)
    ax6.axvline(3*pi/2, linestyle='--', color='gray', lw=1)
    ax6.axvline(2*pi  , linestyle='-' , color='gray', lw=1)
    ax6.axvline(5*pi/2, linestyle='--', color='gray', lw=1)
    ax6.legend(loc='upper right', fontsize='small')
    
    fig.tight_layout()
    if save_image and niter != 0:
        fig.savefig(path_prefix+'/{:04d}.png'.format(niter//disp_modulo))
    fig.canvas.draw_idle()
    
    mpl_pause_background(0.00001)

for k in range(0):
    do_plot(False)

In [12]:

def FD_2D_ILapAdv_matrix (Nx_phys, Ny_phys, Δx, Δy, Δt, µ, ρ, ρ_next, u, v, BCdir_left=True, BCdir_right=True, BCdir_top=True, BCdir_bot=True):

	from cfdutils import ffi, lib as cfdutils_lib
	row = np.zeros(5*Nx_phys*Ny_phys, dtype=np.uint32)
	col = np.zeros(5*Nx_phys*Ny_phys, dtype=np.uint32)
	val = np.zeros(5*Nx_phys*Ny_phys, dtype=np.float64)
	cfdutils_lib.build_FD_2D_ILapAdv_centered_matrix(Nx_phys, Ny_phys, Δx, Δy, ffi.cast("double*",ρ.ctypes.data), ffi.cast("double*",ρ_next.ctypes.data), ffi.cast("double*",u.ctypes.data), ffi.cast("double*",v.ctypes.data), µ, Δt, BCdir_left, BCdir_right, BCdir_top, BCdir_bot, ffi.cast("uint32_t*",row.ctypes.data), ffi.cast("uint32_t*",col.ctypes.data), ffi.cast("double*",val.ctypes.data))
	
	ILapAdv = sp.csc_matrix((val,(row,col)), shape=(Nx_phys*Ny_phys,Nx_phys*Ny_phys))
	LU_decomp = lg.splu(ILapAdv,)

	# Solver function A.X = RHS
	def ILapAdv_solver (RHS, BCdir_val_left=None, BCdir_val_right=None, BCdir_val_top=None, BCdir_val_bot=None):
		# include Dirichlet boundary conditions in the RHS
		RHS_BC = np.copy(RHS)
		if BCdir_val_left is not None:
			if not BCdir_left: raise ValueError("Can't set left boundary value if the matrix is not constructed with left Dirichlet BC")
			RHS_BC[0,:] += BCdir_val_left * ( 2*µ / Δx**2 + ρ_next[1,1:-1] * u[1,1:-1] / Δx )
		if BCdir_val_right is not None:
			if not BCdir_right: raise ValueError("Can't set right boundary value if the matrix is not constructed with right Dirichlet BC")
			RHS_BC[-1,:] += BCdir_val_right * ( 2*µ / Δx**2 - ρ_next[-2,1:-1] * u[-2,1:-1] / Δx )
		if BCdir_val_top is not None:
			if not BCdir_top: raise ValueError("Can't set top boundary value if the matrix is not constructed with top Dirichlet BC")
			RHS_BC[:,0] += BCdir_val_top * ( 2*µ / Δy**2 - ρ_next[1:-1,1] * v[1:-1,1] / Δy )
		if BCdir_val_bot is not None:
			if not BCdir_bot: raise ValueError("Can't set bottom boundary value if the matrix is not constructed with bottom Dirichlet BC")
			RHS_BC[:,-1] += BCdir_val_bot * ( 2*µ / Δy**2 + ρ_next[1:-1,-2] * v[1:-1,-2] / Δy )
		# flatten
		f2 = RHS_BC.ravel()
		# solve
		X = LU_decomp.solve(f2)
		return X.reshape(RHS.shape)

	return ILapAdv, ILapAdv_solver


In [13]:
while loop_continue:

    def VelocityGhostPoints(u,v):
        ### left
        u[0, :] = -u[1, :]
        v[0, :] = v[1, :]
        ### right    
        u[-1, :] = -u[-2, :]
        v[-1, :] = v[-2, :]
        ### bottom
        u[:,  0] = u[:,  1]  # free slip
        v[:,  0] = -v[:,  1]  # imp.
        ### top     
        u[:, -1] = u[:, -2]
        v[:, -1] = -v[:, -2]
    
    VelocityGhostPoints(u,v)
    
    if ρ_semilag2_coeff is None:
        ρ_next = SemiLag(u,v, ρ, Δx,Δy,Δt)
    else:
        ρ_next = SemiLag2(u,v, ρ, Δx,Δy,Δt, ρ_semilag2_coeff)
    
    ρ_next[ 0, :] = 2 * (1) - ρ_next[ 1, :]
    ρ_next[-1, :] = 2 * (1) - ρ_next[-2, :]
    ρ_next[:,  0] = 2 * (1) - ρ_next[:,  1]
    ρ_next[:, -1] = 2 * (1) - ρ_next[:, -2]
    
    fx,fy = force(xx, yy, t+Δt)
    
    rhsx = ρ[1:-1, 1:-1] * u[1:-1, 1:-1] /Δt  - µ * (s[2:,1:-1]-s[:-2,1:-1])/Δx/2  + np.transpose(fx)
    rhsy = ρ[1:-1, 1:-1] * v[1:-1, 1:-1] /Δt  - µ * (s[1:-1,2:]-s[1:-1,:-2])/Δy/2  + np.transpose(fy)

    ILapAdv, ILapAdv_solver = FD_2D_ILapAdv_matrix (Nx_phys, Ny_phys, Δx, Δy, Δt, µ, ρ, ρ_next, u, v, BCdir_left=True, BCdir_right=True, BCdir_top=False, BCdir_bot=False)
    ustar[1:-1, 1:-1] = ILapAdv_solver(rhsx, BCdir_val_left=0, BCdir_val_right=0)
    ILapAdv, ILapAdv_solver = FD_2D_ILapAdv_matrix (Nx_phys, Ny_phys, Δx, Δy, Δt, µ, ρ, ρ_next, u, v, BCdir_left=False, BCdir_right=False, BCdir_top=True, BCdir_bot=True)
    vstar[1:-1, 1:-1] = ILapAdv_solver(rhsy, BCdir_val_top=0, BCdir_val_bot=0)
    
    VelocityGhostPoints(ustar,vstar)

    divstar = Divergence(ustar,vstar, Δx,Δy)

    retry = True
    while retry:
        try:
            DρD,DρD_solver = FD_2D_DρD_matrix(Nx_phys, Ny_phys, Δx, Δy, ρ_next, BCdir_left=False, BCdir_right=False, BCdir_top=False, BCdir_bot=False, backend_build_C=True)
            retry = False
        except RuntimeError:
            ρ_next += np.random.normal(0,1e-10,size=(Nx,Ny))

    phi[1:-1, 1:-1] = DρD_solver(RHS=divstar[1:-1,1:-1]/Δt)
    phi[0   ,  :  ] = + phi[ 1   ,  :  ]
    phi[  -1,  :  ] = + phi[-2   ,  :  ]
    phi[ :  ,   -1] = + phi[  :  ,   -2]
    phi[ :  , 0   ] = + phi[  :  ,    1]

    dx_phi[1:-1, :] = (phi[2:, :] - phi[:-2, :])/Δx/2
    dy_phi[:, 1:-1] = (phi[:, 2:] - phi[:, :-2])/Δy/2

    u = ustar - Δt*dx_phi / ρ_next
    v = vstar - Δt*dy_phi / ρ_next

    s[1:-1, 1:-1] -= divstar[1:-1, 1:-1]

    ρ[:] = ρ_next[:]
    
    if niter%disp_modulo == 0:
        do_plot(save_image=True)
    
    t += Δt
    niter += 1